# Read Parquet files using Python

Dependencies:
* pyarrow
* fsspec
* s3fs

In [ ]:
#import pandas as pd
import boto3

: 

In [ ]:
session = boto3.Session(profile_name='personal')
s3 = session.resource('s3')

#### Settings

In [ ]:
# Source
BUCKET = 'sorel-20m'
PREFIX = '09-DEC-2020/binaries'
BUCKET_PATH = f's3://{BUCKET}'
SRC_PATH = f'{BUCKET_PATH}/{PREFIX}'

filter_chars = '0000'
#filter_chars = '0'

# Target
TARG_BUCKET = 'sorel-20m-demo'
TARG_BUCKET_PATH = f's3://{TARG_BUCKET}'
TARG_PREFIX = 'output'
TARG_PATH = f'{TARG_BUCKET_PATH}/{TARG_PREFIX}'

MB = 1024 * 1024


#### Read files created

In [ ]:
targ_bucket = s3.Bucket(TARG_BUCKET)
parquet_files = targ_bucket.objects.filter(Prefix=TARG_PREFIX)

num_found = 0
tot_size = 0
for i, o in enumerate(parquet_files):
    if i < 5:
        print(o.key, o.size)
    num_found += 1
    tot_size += o.size
if i >= 5:
    print('...')
print(f'Found {num_found} files.  Tot size = {tot_size / MB} MB.  Avg size = {tot_size / num_found / MB} MB')

output/00/part-00000-fe7492a2-7bbe-4729-a241-2b226a193df9-c000.snappy.parquet 304919267
output/00/part-00001-fe7492a2-7bbe-4729-a241-2b226a193df9-c000.snappy.parquet 299356093
output/00/part-00002-fe7492a2-7bbe-4729-a241-2b226a193df9-c000.snappy.parquet 258069863
output/00/part-00003-fe7492a2-7bbe-4729-a241-2b226a193df9-c000.snappy.parquet 177790742
output/00/part-00004-fe7492a2-7bbe-4729-a241-2b226a193df9-c000.snappy.parquet 171920540
...
Found 23596 files.  Tot size = 566191.5723371506 MB.  Avg size = 23.99523530840611 MB


In [ ]:
# Too slow....
# tot_pe_files = 0
# for o in parquet_files:
#     df_parquet = pd.read_parquet(f'{TARG_BUCKET_PATH}/{o.key}', storage_options=dict(profile='personal'))
#     num_pe_files = len(df_parquet)
#     tot_pe_files += num_pe_files
#     print(f'{o.key} has {num_pe_files} files.  Size = {o.size}.')

athena_client = boto3.client('athena')
queryStart = athena_client.start_query_execution(
    QueryString = 'SELECT count(*) num_pe, sum(length) tot_size FROM PE_BIN',
    QueryExecutionContext = {
        'Catalog': 'AWSDataCatalog',
        'Database': 'sorel'
    },
    #WorkGroup='sorel'
    ResultConfiguration = { 'OutputLocation': 's3://sorel-20m-demo/tmp/athena/tmp_q01'}
)


: 

In [20]:
queryExecution = athena_client.get_query_execution(QueryExecutionId=queryStart['QueryExecutionId'])

In [21]:
queryExecution

{'QueryExecution': {'QueryExecutionId': 'e122912c-b35f-4308-a3da-294847e2f494',
  'Query': 'SELECT count(*) num_pe, sum(length) tot_size FROM PE_BIN',
  'StatementType': 'DML',
  'ResultConfiguration': {'OutputLocation': 's3://sorel-20m-demo/tmp/athena/tmp_q01/e122912c-b35f-4308-a3da-294847e2f494.csv'},
  'QueryExecutionContext': {'Database': 'sorel', 'Catalog': 'awsdatacatalog'},
  'Status': {'State': 'FAILED',
   'StateChangeReason': 'SYNTAX_ERROR: line 1:51: Schema sorel does not exist',
   'SubmissionDateTime': datetime.datetime(2023, 3, 19, 23, 51, 15, 492000, tzinfo=tzlocal()),
   'CompletionDateTime': datetime.datetime(2023, 3, 19, 23, 51, 16, 6000, tzinfo=tzlocal()),
   'AthenaError': {'ErrorCategory': 2,
    'ErrorType': 1301,
    'Retryable': False,
    'ErrorMessage': 'SYNTAX_ERROR: line 1:51: Schema sorel does not exist'}},
  'Statistics': {'EngineExecutionTimeInMillis': 277,
   'DataScannedInBytes': 0,
   'TotalExecutionTimeInMillis': 514,
   'QueryQueueTimeInMillis': 193,

In [9]:
df_parquet.columns

Index(['path', 'modificationTime', 'length', 'content'], dtype='object')

In [ ]:
import sorel.shared.ls_sorel_bin as ls_sb

num_files_from_s3, file_size_s3 = ls_sb.main()

ModuleNotFoundError: No module named 'ls_sorel_bin_multiproc'

In [ ]:
print(f'Num PE files contained in parquet files = {tot_pe_files}')
print(f'Num parquet files = {num_found}.  Avg # of PE files/parquet file = {tot_pe_files / num_found}')
print(f'Avg PE file size = {tot_size / tot_pe_files} bytes')

Num PE files contained in parquet files = 2377
Num parquet files = 92.  Avg # of PE files/parquet file = 25.83695652173913
Avg PE file size = 998473.5717290703 bytes


In [ ]:
# TODO Implement this for a selected file

# print(f'Detail for parquet file {o.key}')
# num_pe_files = 0
# tot_size_cur = 0
# for row in df_parquet.itertuples():
#     num_pe_files += 1
#     tot_size_cur += row.length
#     print(row.path, row.modificationTime, row.length)

Detail for parquet file output/fff/part-00091-788e433a-08e2-45fb-89db-13a511632cc3-c000.snappy.parquet
s3://sorel-20m/09-DEC-2020/binaries/fff224717c551708c7025b3c6150fb6b0716e6df686436fb94b2fe5a5ee157c4 2020-12-03 09:52:10 3408
s3://sorel-20m/09-DEC-2020/binaries/fffa8156c48752b5967cdb8955babcf5e8db8809589188412bf21b7b47a2f299 2020-12-03 09:54:02 3322
s3://sorel-20m/09-DEC-2020/binaries/ffff7fc2b23ce8832ec10ecdea8da3c7ffbeedece07864460f36d0546e5a9226 2020-12-03 09:55:24 3170
s3://sorel-20m/09-DEC-2020/binaries/fff96d12e2e3be66caacf2e1299d4c8a553b40ffc07fac763aba362dca6186cb 2020-12-03 09:53:48 3050
s3://sorel-20m/09-DEC-2020/binaries/fffe182691b22f7ad1fa415527eb3d0d030d12b62653bb5cc8b1585cd750ea1d 2020-12-03 09:55:06 3050
s3://sorel-20m/09-DEC-2020/binaries/fff353f4b43ef0a1c88efd383ce73857d6768a4697768d860f2d39b5f52b3524 2020-12-03 09:52:24 2988
s3://sorel-20m/09-DEC-2020/binaries/fff0b0c5f007abf2c1bc2c3e873f45eb0359fef364a369c0b8df246d41c4aa6c 2020-12-03 09:51:53 2850
s3://sorel-20m/

In [ ]:
print(f'In this parquet file, found {num_pe_files} PE files.  Size of PE files = {tot_size_cur}.  Size of parquet file = {o.size}.')

In this parquet file, found 24 PE files.  Size of PE files = 49944.  Size of parquet file = 55038.


Manually verified that the size of the PE files matches source